# 🪣 Working with Amazon S3 Object Storage

This lab demonstrates how to work with **Amazon S3 (Simple Storage Service)** using both the **AWS Management Console** and programmatically via the **Boto3 SDK**.  
You will create an S3 bucket, upload and query structured and semi-structured data, configure permissions and versioning, and automate tasks with reusable Python functions.
You can find all the code examples about interaction with S3 in Boto3 [Documentations](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-examples.html)

By completing this lab, you will be able to:

- Create and configure an **S3 bucket** using Terraform or the AWS console.  
- Upload **structured (CSV)**, **semi-structured (JSON)**, and **unstructured (image)** data to Amazon S3.  
- Interact with S3 objects programmatically using **Boto3**.  
- Configure **bucket access policies** and **public access settings**.  
- Enable and inspect **versioning** in S3.  
- Clean up and **delete buckets** and objects, including versioned ones.

The required data are located in **data** folder

## 1. Insttall the required Packages


In [ ]:
%pip install boto3
%pip install python-dotenv boto3
%pip install -q pandas s3fs python-dotenv


# 1. Import the required liberaries 

In [2]:
import boto3
import json
import os
from dotenv import load_dotenv
from pathlib import Path
import boto3
import mimetypes

from typing import Any, Dict
from IPython.display import HTML

## ⚙️ 1. Setting Up AWS Credentials

Before running any S3 operations, configure your AWS credentials securely.  
We store credentials in a `.env` file and create a **global session** that all Boto3 clients will use automatically.

In [3]:

# Force .env to override anything already set in your terminal

# Load and clean .env
load_dotenv(".env", override=True)

def clean(v): 
    return v.strip().strip('"').strip("'") if v else v

AWS_ACCESS_KEY_ID     = clean(os.getenv("AWS_ACCESS_KEY_ID"))
AWS_SECRET_ACCESS_KEY = clean(os.getenv("AWS_SECRET_ACCESS_KEY"))
AWS_DEFAULT_REGION    = clean(os.getenv("AWS_DEFAULT_REGION")) or "us-west-2"

# Create a global session (used automatically by all boto3 clients)
boto3.setup_default_session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION,
)

print(" Global AWS session configured for", AWS_DEFAULT_REGION)



 Global AWS session configured for us-west-2


# 4. Uploading Files to S3


The following reusable function uploads any file from your local environment to an S3 bucket.

In [4]:


def upload_file_to_s3(local_file_path: str, bucket_name: str, object_key: str) -> None:
    """
    Uploads a local file to S3 using boto3 with automatic content type detection.

    Args:
        local_file_path (str): Path to the local file to upload.
        bucket_name (str): Name of the target S3 bucket.
        object_key (str): S3 object key (path/name in the bucket).
    """
    local_file = Path(local_file_path)
    if not local_file.exists():
        raise FileNotFoundError(f"Local file not found: {local_file_path}")

    # Detect MIME type for correct Content-Type metadata
    ctype, _ = mimetypes.guess_type(local_file.name)
    extra = {"ContentType": ctype} if ctype else {}

    # Build the S3 client
    s3_client = boto3.client("s3")

    try:
        s3_client.upload_file(str(local_file), bucket_name, object_key, ExtraArgs=extra)
        print(f"Uploaded {local_file} → s3://{bucket_name}/{object_key}")
    except Exception as e:
        print(f"Error uploading file to S3: {e}")


# a. Upload the Structured File
Upload your csv file into the bucket.


In [5]:
# Define the local file path, and object key
local_file_path = 'data/csv/ratings_ml_training_dataset.csv'
object_key = 'csv/ratings_ml_training_dataset.csv'
BUCKET_NAME='example-bucket-d2e948dbd0db'

# Upload the file to S3
upload_file_to_s3(local_file_path, BUCKET_NAME, object_key)

Uploaded data\csv\ratings_ml_training_dataset.csv → s3://example-bucket-d2e948dbd0db/csv/ratings_ml_training_dataset.csv


In [ ]:

# check the uploaded file in the bucket

!aws s3 ls $BUCKET_NAME/csv/

## Query the the uploaded csv
You can read structured data directly from S3 into a pandas DataFrame

In [ ]:

load_dotenv(".env", override=True)

import os, pandas as pd

BUCKET = "example-bucket-d2e948dbd0db"   
KEY    = "csv/ratings_ml_training_dataset.csv"

s3_path = f"s3://{BUCKET}/{KEY}"
df = pd.read_csv(s3_path)

# optional: tidy column names
df.columns = [c.strip().lower() for c in df.columns]
print("Rows:", len(df), "Cols:", len(df.columns))
df.head()


In [ ]:
# Example: rows for France with quantityinstock > 5000, select a few columns
q1 = (
    df.loc[(df["country"] == "France") & (df["quantityinstock"] > 5000),
           ["city", "country", "productcode", "productline", "quantityinstock", "buyprice"]]
    .sort_values("quantityinstock", ascending=False)
)
q1.head(10)


In [ ]:
# 1) stock by country
stock_by_country = (
    df.groupby("country", as_index=False)["quantityinstock"].sum()
    .sort_values("quantityinstock", ascending=False)
)

# 2) average price by product line
avg_prices = (
    df.groupby("productline", as_index=False)
      .agg(avg_buyprice=("buyprice", "mean"), avg_msrp=("msrp", "mean"))
      .sort_values("avg_buyprice", ascending=False)
)

# show results
display(stock_by_country.head(10))
display(avg_prices.head(10))


## b. Upload the semi-structured file using the pere-built function
Now upload semi-structured data (a JSON file)

In [6]:
# Define the local file path, and object key
local_file_path = 'data/json/delivery-stream-one-record.json'
object_key = 'json/delivery-stream-one-record.json'
BUCKET_NAME='example-bucket-d2e948dbd0db'

# Upload the file to S3
upload_file_to_s3(local_file_path, BUCKET_NAME, object_key)

Uploaded data\json\delivery-stream-one-record.json → s3://example-bucket-d2e948dbd0db/json/delivery-stream-one-record.json


In [ ]:
!aws s3 ls $BUCKET_NAME/json/

## 5. Downloading Files from S3
You can download files locally with another reusable function.

In [ ]:


def download_file_from_s3(bucket_name: str, object_key: str, local_file_path: str) -> None:
    """
    Downloads an object from S3 using the existing authenticated `s3` client.
    """
    dest = Path(local_file_path)
    dest.parent.mkdir(parents=True, exist_ok=True)

    s3_client = boto3.client("s3")


    try:
        s3_client.download_file(bucket_name, object_key, str(dest))
        print(f"Downloaded s3://{bucket_name}/{object_key} → {dest}")
    except Exception as e:
        print(f"Error downloading from S3: {e}")


In [ ]:
# Define local destination paths for csv file
csv_local = "./downloads/ratings_ml_training_dataset.csv"
json_local = "./downloads/delivery-stream-one-record.json"
BUCKET='example-bucket-d2e948dbd0db'

download_file_from_s3(BUCKET, json_key, json_local)


In [ ]:

# Define local destination paths for json file
csv_key = "csv/ratings_ml_training_dataset.csv"
json_key = "json/delivery-stream-one-record.json"
BUCKET_NAME='example-bucket-d2e948dbd0db'


download_file_from_s3(BUCKET_NAME, csv_key, csv_local)

# c. Upload the Unstructured file 
This functions uploads the image file into the s3 bucket

In [ ]:
local_file_path_image_v1 = 'data/images/v1/s3_logo.png'
object_key_image = 'images/s3_logo.png'

upload_file_to_s3(local_file_path_image_v1, BUCKET_NAME, object_key_image)

In [ ]:
#check the folder in the bucket
!aws s3 ls $BUCKET_NAME/images/

Now you can also check the uploaded folders and files in the bucket in AWS Console

![S3 Architecture](assets/img2.png)

## 6. Managing Bucket Access Settings

Amazon S3 buckets are **private by default**, meaning that only the AWS account owner and explicitly authorized users or roles can access them.  
To securely share data through web browser or make certain objects public (for example, images or static web files), you must configure **bucket access settings** properly

In [8]:
def s3_public_access_setup(bucket_name: str, public_access_block_configuration: Dict[str, Any]) -> None:
    """Sets public access configuration for S3 bucket

    Args:
        bucket_name (str): Bucket name
        public_access_block_configuration (Dict[str, Any]): Configuration for public access
    """
    
    s3_client = boto3.client("s3")
    
    # Update the bucket's public access settings
    s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration=public_access_block_configuration
    )

### 🧭 Understanding Public Access Configuration

S3 provides a set of **public access block settings** that act as a safety layer, preventing accidental exposure of sensitive data. These settings can be enabled or disabled at the bucket level.

 **Caution**: Disabling public access protection makes your bucket publicly readable. Use only in a safe, temporary test environment.

In [9]:
# Define the public access settings  
public_access_configuration = {
    'BlockPublicAcls': False,
    'IgnorePublicAcls': False,
    'BlockPublicPolicy': False,
    'RestrictPublicBuckets': False
}

s3_public_access_setup(bucket_name=BUCKET_NAME, 
                       public_access_block_configuration=public_access_configuration)

## 7. Setting a Bucket Policy
Define a policy that controls permissions for specific prefixes or users.

In [10]:
def s3_put_bucket_policy(bucket_name: str, policy: Dict[str, Any]) -> None:
    """Allows to put bucket policies

    Args:
        bucket_name (str): Bucket name
        policy (Dict[str, Any]): Bucket policy
    """
    
    s3_client = boto3.client("s3")
    
    response = s3_client.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))
    return response

**Example**: Make all image files in the bucket publicly readable

In [11]:
policy = { 
    "Version": "2012-10-17", 
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{BUCKET_NAME}/images/*"
        }
    ]
}

In [12]:
response = s3_put_bucket_policy(bucket_name=BUCKET_NAME, policy=policy) 
print(response)

{'ResponseMetadata': {'RequestId': 'YHK865GHST575M0K', 'HostId': 'dmAEelvLiZxsGIR9mhmiqR8etTOAs6utvDqBDMdO7BZNPB7FgZdcb7iXmhoYQ86JMtcdkCTI/4dY+UuV2BvaXt6bEi6vzXatY5UCFRQALdY=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'dmAEelvLiZxsGIR9mhmiqR8etTOAs6utvDqBDMdO7BZNPB7FgZdcb7iXmhoYQ86JMtcdkCTI/4dY+UuV2BvaXt6bEi6vzXatY5UCFRQALdY=', 'x-amz-request-id': 'YHK865GHST575M0K', 'date': 'Wed, 12 Nov 2025 11:23:24 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


Now you can access the image file from the browser by copying its Object Url from AWS Consol

![S3 Architecture](assets/img4.png)


## 8. Enabling Object Versioning
Amazon S3 **Versioning** is a powerful feature that allows you to **store multiple versions of the same object** within a bucket.  
When versioning is enabled, S3 keeps track of every modification and deletion of your objects, providing a built-in mechanism for **data protection, recovery, and auditability**.
By default, an S3 bucket keeps only the latest version of each object. If you upload a new file with the same name, the previous file is **overwritten permanently**.

With versioning **enabled**, each time you upload a new version of a file with the same key (name), S3 automatically assigns it a **unique version ID** and preserves the older copies.  
You can retrieve or restore any previous version at any time. You can manage versioning at the bucket level using the **`put_bucket_versioning()`** API call.  
Below is the Python function to enable or suspend versioning using Boto3

In [ ]:
def configure_bucket_versioning(bucket_name: str, versioning_config: Dict[str, str]) -> Dict[Any, Any]:
    
    s3_client = boto3.client("s3")
    
    # Enable bucket versioning
    response = s3_client.put_bucket_versioning(
        Bucket=bucket_name,
        VersioningConfiguration=versioning_config
    )

    return response

In [ ]:
versioning_config = {'Status': 'Enabled'}

response = configure_bucket_versioning(bucket_name=BUCKET_NAME, 
                                       versioning_config=versioning_config)
print(response)

In [ ]:
local_file_path_image_v2 = 'data/images/v2/s3_logo.png'
object_key_image = 'images/s3_logo.png'

upload_file_to_s3(local_file_path_image_v2, BUCKET_NAME, object_key_image)

## 9. Listing Object Versions
You can retrieve all versions of objects under a specific prefix.

In [ ]:
def list_object_versions(bucket_name: str, prefix_key: str) -> None:
    # Create an S3 client
    
    s3_client = boto3.client("s3")

    # List object versions
    response = s3_client.list_object_versions(Bucket=bucket_name, Prefix=prefix_key)

    # Process the response to get object versions
    for version in response.get('Versions', []):
        print("Object Key:", version['Key'])
        print("Object Version Id:", version['VersionId'])
        print("Is Latest:", version['IsLatest'])
        print("Last Modified:", version['LastModified'])
        print()

list_object_versions(bucket_name=BUCKET_NAME, prefix_key='images/')

## 10. Cleaning Up — Deleting Buckets and Objects
When you finish testing, you can delete all objects (including versions) and remove the bucket.

In [ ]:
def s3_delete_bucket(bucket_name: str, delete_objects: bool) -> Dict[Any, Any]:
    s3_client = boto3.client('s3')
    
    if delete_objects:
        # List all versions of all objects in the bucket
        response = s3_client.list_object_versions(Bucket=bucket_name)
        
        # Delete all object versions
        for version in response.get('Versions', []):
            key = version['Key']
            version_id = version['VersionId']
            s3_client.delete_object(Bucket=bucket_name, Key=key, VersionId=version_id)
        
        # Delete all delete markers
        for delete_marker in response.get('DeleteMarkers', []):
            key = delete_marker['Key']
            version_id = delete_marker['VersionId']
            s3_client.delete_object(Bucket=bucket_name, Key=key, VersionId=version_id)        
    
    # Delete the bucket
    response = s3_client.delete_bucket(
        Bucket=bucket_name
    )

    return response

response = s3_delete_bucket(bucket_name=BUCKET_NAME, delete_objects=True)
print(response)